In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Aug  3 14:53:49 2018

@author: madeleineernst
"""

In [ ]:

#############################################################################################################################
#                                                                                                                           #
#      Create motif feature table with count overlap scores  (nr. of molecules per motif) and a selected chemical class     #
#                                                                                                                           #
#############################################################################################################################

import pandas as pd

subcl = pd.read_csv('ClassyFire_Output_forCytoscape.tsv',sep='\t')

motifs = pd.read_csv('Mass2Motifs_Nodes.csv',sep=';')
motifs = motifs.rename(columns = {'shared.name':'cluster.index'})
motifs.index = motifs['cluster.index']

ft = pd.read_csv('/FeatureTable_Rhamnaceae.csv',index_col=0, sep=',') 
ft[ft < 1000] = 0
ft = ft.loc[ft.index.isin(subcl['cluster.index'])]

# create feature dictionary
ind = ft.index
bt = ft.apply(lambda x: x > 0)
ions = bt.apply(lambda x: list(ind[x.values]), axis=0).to_dict()
ions_o = bt.apply(lambda x: list(ind[x.values]), axis=0).to_dict()

# create motif dictionary
motdic = motifs['Motif'].to_dict()

# replace values in feature dictionary with values in motif dictionary

for k in ions:
    ions[k] = [motdic.get(item,item)  for item in ions[k]] # replace all features with corresponding motifs

for k in ions:
    ions[k] = [x for x in ions[k] if not isinstance(x, int)] # remove features with no motifs (integers)

for k in ions:
    ions[k] = [i.split(',') for i in ions[k]] # string split all motifs to single values
    
for k in ions:    
    ions[k] = [j for i in ions[k] for j in i] # merge list of lists into one list
    
for k in ions:
    ions[k] = list(set(ions[k])) # remove duplicate motifs

# convert dictionary to binary pandas dataframe

for k in ions:
    l = ions[k]
    f = motifs.loc[motifs['cluster.index'].isin(ions_o[k])]
    o = []
    for j in range(0,len(l)):
        o.append(f[l[j]][f[l[j]]>0.3].count())
    ions[k] = dict(zip(ions[k], o))
    
motifdat = pd.get_dummies(pd.DataFrame.from_dict(ions,orient='index')).groupby(by=lambda x: '_'.join(x.split('_')[1:]), axis=1).sum()
motifdat = motifdat.transpose()

motifdat.to_csv("motiftable_OverlapScores_Count_cutoff1000.tsv",sep='\t',index=True)